In [0]:
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import mnist, cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils
from keras.regularizers import l2
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
from PIL import Image

## Model check point
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
# filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

## File path to read data
mypath ='./training50classes/'
mypath2 ='./validation50classes/'

## Model
batch_size = 32
nb_classes = 50
nb_epoch = 200
data_augmentation = True

# sketch-A-Net
# apply a 3x3 convolution with 64 output filters on a 256x256 image:
model = Sequential()
model.add(Convolution2D(64, 15, 15, border_mode='same', input_shape=(225, 225, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Convolution2D(128, 5, 5, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Convolution2D(256, 3, 3, border_mode='same'))
model.add(Activation('relu'))

model.add(Convolution2D(256,3,3, border_mode='same'))
model.add(Activation('relu'))

model.add(Convolution2D(256,3,3, border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Convolution2D(512,7,7, border_mode='same'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Convolution2D(512,1,1, border_mode='same'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(250,1,1, border_mode='same'))
model.add(Flatten())
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
print('Using real-time data augmentation.')

# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    samplewise_center=True,  # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=True,  # divide each input by its std
    zca_whitening=True,  # apply ZCA whitening
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images

validation_datagen = ImageDataGenerator()

train_generator = datagen.flow_from_directory(
    mypath,  # this is the target directory
    target_size=(225,225),  # all images will be resized to 150x150
    batch_size=batch_size,
    shuffle = True,
    color_mode='grayscale',
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
       mypath2,  # this is the target directory
       target_size=(225,225),  # all images will be resized to 150x150
       batch_size=batch_size,
       shuffle = True,
       color_mode='grayscale',
       class_mode='categorical')

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(train_generator,
                    samples_per_epoch=20000,
                    nb_epoch=nb_epoch,
                    callbacks=callbacks_list, 
                    validation_data = validation_generator,
                    nb_val_samples=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (15, 15), input_shape=(225, 225,..., padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (7, 7), padding="same")`


Using real-time data augmentation.
Found 3000 images belonging to 50 classes.
Found 1000 images belonging to 50 classes.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., callbacks=[<keras.ca..., validation_data=<keras_pre..., steps_per_epoch=625, epochs=200, validation_steps=2)`


Epoch 1/200


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


625/625 [==============================] - 148s 236ms/step - loss: 15.7827 - acc: 0.0197 - val_loss: 15.8663 - val_acc: 0.0156

Epoch 00001: val_acc improved from -inf to 0.01562, saving model to weights-improvement-01-0.02.hdf5
Epoch 2/200
625/625 [==============================] - 142s 228ms/step - loss: 15.7909 - acc: 0.0203 - val_loss: 15.6144 - val_acc: 0.0312

Epoch 00002: val_acc improved from 0.01562 to 0.03125, saving model to weights-improvement-02-0.03.hdf5
Epoch 3/200
625/625 [==============================] - 146s 233ms/step - loss: 15.7992 - acc: 0.0198 - val_loss: 15.8663 - val_acc: 0.0156

Epoch 00003: val_acc did not improve from 0.03125
Epoch 4/200
625/625 [==============================] - 143s 229ms/step - loss: 15.7949 - acc: 0.0201 - val_loss: 15.8663 - val_acc: 0.0156

Epoch 00004: val_acc did not improve from 0.03125
Epoch 5/200
625/625 [==============================] - 146s 233ms/step - loss: 15.7917 - acc: 0.0203 - val_loss: 15.1107 - val_acc: 0.0625

Epoch 0

KeyboardInterrupt: ignored